# Jupyter notebook in Scispot

- Scispot --> LabSheets --> Jupyter --> +New --> write script/notebook
  - ~~need jupyterhub server credentials?~~

- ? can I look up a UUID by sample name?
  - update-by-id to fetch UUID from Sample ID
- ? can I look up UUIDs by manifest name?
  - not yet! 
- ? can I specify other "variable" inputs/params? or can scientist edit the ipynb with those values?
  - A: yes, can code input parameters 
  - ? can I read in other sheets into pandas?
    - ~~nm don't worry about this for now...~~ done, see below


### tips
- update-by-id to fetch UUID from Sample ID 
- https://www.loom.com/share/32c01c2cb1294720b15596b0a43af5e9

### notes 
- Sample IDs from 96-well manifest export → Protein Manager registrations
- Protein Manager registrations completed for following Sample manifests:
  - ChESS_dbgen1_plate1
  - ChESS_dbgen1_202212_plate1rep2 
  - ChESS_dbgen1_202212_plate1rep3
  - ... how to stage these in the future? LC-MS queue?

### todo
- Parent-Sample and Child-Protein UUID links need to be updated for historical Benchling protein manager entries
- Related, Update programmatic entries with child linkages (i.e. update Sample Manager entries with Child links?)
- Feature “Request”: Protein Manager > Peptide Manager 
- uuids that generated from buggy add_entry to protein manager:
  - 0344a99d-888a-443e-9a23-13eceb279ae1 - Sample Manager and Protein Manager and Peptide Manager and ...??
  - 90b19820-726d-4086-b74a-199d37899ea5 - Sample Manager and Protein Manager and Peptide Manager and ...??
  - 98f8e123-84ab-4171-b26e-6c3c0ad6ecf3 - Sample Manager and Protein Manager and Peptide Manager and ...??
  - f83d6043-0901-4362-bef9-324987e06451 - doesn't exist
  - 485546de-f451-46cd-afbb-7bad60e998d5
  - e4695003-2074-458c-8080-9a659bc208c5
- Pull a plate of samples with some manifest name/location from API? (not yet, per satya)


In [1]:
## script to generate Scispot registrations for ChESS 96-well plate
## lkp 2022/11/15

import pandas as pd
import requests
import json
import sys
import datetime

sys.stdout.write("Imported required packages")

api_key_file = 'G:/My Drive/Lindsay Pino/proj/2023_scispot_utils/data/scispot_api_key.txt'
API_KEY = open(api_key_file, 'r').readlines()[0]
#print(API_KEY)

Imported required packages

In [5]:
## functions

# lookup an entry by UUID
def find_entry_from_uuid(manager, uuid_list):
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/find-row"
    payload = {
        "apiKey": API_KEY,
        "manager": manager,
        "uuid": uuid_list
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

# turn json Scispot return into a pandas df
def json_to_pd_df(json_in):
    df_out = pd.DataFrame(data=json_in['row']).T
    df_out.columns = list(json_in['headers'])
    return df_out

# fetch any row from the Protein Manager to get the columns
def fetch_random_entry():
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/list-rows"
    payload = {
        "apiKey": API_KEY,
        "manager": "Peptide Manager",
        "pageSize": "1",
        "page": "1"
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

# fetch a Scispot entry based on the Sample ID
def fetch_entry_from_id(manager, sample_id):
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/find-row-by-id"
    payload = {
        "apiKey": API_KEY,
        "labsheet": manager,
        "id": sample_id
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

sys.stdout.write("Imported required functions")

Imported required functions

In [ ]:
## examples

# look up an entry based on UUID
test_uuid = "0344a99d-888a-443e-9a23-13eceb279ae1"
find_entry_from_uuid('Sample Manager', test_uuid)

# make the scispot return a pretty dataframe
json_to_pd_df(find_entry_from_uuid('Sample Manager', test_uuid))

# initialize an empty Manager
prot_df = pd.DataFrame(data=fetch_random_entry()['rows'], columns=fetch_random_entry()['headers'])
prot_df= prot_df.drop(prot_df.index[0])
print(prot_df)

# test_sampleid = "THP1_dbgen1_VTP50469_3"
fetch_entry_from_id("Sample Manager",test_sampleid)

In [4]:
# add a new row to the Protein Manager
def add_protein_manager_entry(new_row):
    url = "https://api.scispot.io/tryingtofixcors/labsheets/add-rows"
    payload = {
        "apiKey": API_KEY,
        "manager": "Protein Manager",
        "rows": [new_row]
    }
    headers = {"Content-Type": "application/json"}
    #print(new_row)
    
    response = requests.request("POST", url, json=payload, headers=headers)
    print(response.text)

# generate a new row for the protein manager
def new_protein_row(sample_uuid, fraction):
    
    # generate protein ID from sample ID
    sample_id = json_to_pd_df(find_entry_from_uuid("Sample Manager", sample_uuid))['Sample ID'][0]
    #print(sample_id)
    protein_id = sample_id + "_" + fraction
          
    new_protein_row = [
        protein_id, # Protein ID
        sample_uuid, # Sample ID required, must be UUID
        "e45d66e9-abed-4d24-9dd8-8f074233c2d8", # Protocol ID required, must be UUID
        "01/01/2023", # "Extraction Date" column required, must be MM/DD/YYYY
        "Lindsay", # "Extraction Scientist" column required, must be name
        fraction, # Extraction fraction
        "extractionnote_here", # Extraction Note
        "0", # Protein Concentration (ug/mL)
        "bca_here", # BCA Results
        "", # Location
        "storageform_here", # Storage Form
        "storageamt_here", # Storage Amount (uL)
        "True", # "Pass" column required
        sample_uuid, # Parent UUID
        "", # Children UUID
        "", # Created By
        "" # Creation Date
    ]

    return(new_protein_row)

sys.stdout.write("Imported functions for manipulating Protein Manager")

# add a new row to the Peptide Manager
def add_peptide_manager_entry(new_row):
    url = "https://api.scispot.io/tryingtofixcors/labsheets/add-rows"
    payload = {
        "apiKey": API_KEY,
        "manager": "Peptide Manager",
        "rows": [new_row]
    }
    headers = {"Content-Type": "application/json"}
    #print(new_row)
    
    response = requests.request("POST", url, json=payload, headers=headers)
    print(response.text)

# generate a new row for the peptide manager
def new_peptide_row(protein_uuid):
    
    # generate peptide ID from protein ID
    protein_id = json_to_pd_df(find_entry_from_uuid("Protein Manager", protein_uuid))['Protein ID'][0]
    #print(protein_id)
    peptide_id = protein_id + "_peptide"
          
    new_peptide_row = [peptide_id, # Digestion ID
            protein_uuid, # Protein ID, must be UUID
            '01/01/2023', # Digestion Date 
            'e45d66e9-abed-4d24-9dd8-8f074233c2d8', # Protocol ID (dummy protocol)
            'Lindsay', # Digestion Scientist 
            '', # Protein Input
            'DTT', # Reduction Chemical 
            '', # Reduction Dose
            'IAA', # Alkylation Chemical 
            '', # Alkylation Dose 
            'Trypsin', # Protease Chemical 
            '', # Protease Dose 
            'Formic Acid', # Quench Chemical 
            '', # Quench Dose 
            '', # Storage Location 
            '', # Storage Form 
            '', # Storage Concentration 
            'True', # Pass 
            protein_uuid, # Parent 
            '', # Children 
            '', # Location 
            '' # Created By
    ]

    return(new_peptide_row)

sys.stdout.write("Imported functions for manipulating Peptide Manager")

Imported functions for manipulating Protein Manager

In [ ]:
##
## GENERATE PROTEIN SAMPLES
##

# read in a Scispot plate TSV export of Sample Manager
# ChESS_dbgen1_plate1 was done manually
#sample_fi = "G:/My Drive/Lindsay Pino/proj/2023_scispot_utils/data/Sample-2023-04-10T19_24_09.tsv" # ChESS_dbgen1_202212_plate1rep2 
#sample_fi = "G:/My Drive/Lindsay Pino/proj/2023_scispot_utils/data/Sample-2023-04-10T19_24_09.tsv" # ChESS_dbgen1_202212_plate1rep3
input_df = pd.read_table(sample_fi, sep='\t', engine="python")
input_df.tail()

# generate the new protein samples and add them to SciSpot Protein Manager
# note: uuid header string parses weird > "ï»¿uuid"
for uuid in input_df["ï»¿uuid"]:

    # define Protein fractions to be generated for each Sample
    extraction_frx = ['cytosol',
                     'chromatin',
                     'insoluble']
    
    # generate the Protein Manager row for each fraction
    for frx in extraction_frx:
        new_row = new_protein_row(uuid, frx)
    
        # add the row to the Protein Manager
        #add_protein_manager_entry(new_row)


##
## GENERATE PEPTIDE SAMPLES
##

# read in a Scispot plate TSV export of Protein Manager
protein_fi = "G:/My Drive/Lindsay Pino/proj/2023_scispot_utils/data/Protein-2023-04-11T20_08_57.tsv" # ChESS_dbgen_202212_plate1rep1_chromatin
input_df = pd.read_table(protein_fi, sep='\t', engine="python")
input_df.tail()

# generate the new peptide samples and add them to SciSpot Peptide Manager
# note: uuid header string parses weird > "ï»¿uuid"
for uuid in input_df["ï»¿uuid"]:
    
    # generate the Peptide Manager row
    new_row = new_peptide_row(uuid)
    #print(new_row)

    # add the row to the Peptide Manager
    #add_peptide_manager_entry(new_row)


In [25]:
##
## GENERATE RUN SAMPLES
##

fetch_entry_from_id("Acquisition Manager", "230101_dbgenbacklog")


{'rows': [['c7b2724a-4297-4733-8257-e786ac6253d1',
   '230101_dbgenbacklog',
   '01/01/2023',
   'Daniele',
   'Exploris 480',
   '',
   '',
   '']],
 'headers': ['UUID',
  'Acquisition ID',
  'Acquisition Date',
  'Acquisition Scientist',
  'Instrument',
  'Column ID',
  'Acquisition Notes',
  'Created By'],
 'success': True}

In [26]:
## make manifest
def make_new_plate():
    url = "https://api.scispot.io/tryingtofixcors/manifest/create"
    payload = {
        "apiKey": "f37df3be-56b8-4496-a7e3-3cf2fb5be033",
        "name": "Test Manifest Hello World",
                "template": "96-well plate",
         "plates": [
            {
                "template": "96-well plate",
                "wells": 96,
                "labsheets": [
                    {
                        "labsheet": "Protein Manager",
                        "items": [
                            {
                                "name": "JHC7_dbgen1_DMSO_2_chromatin",
                                "well": "A1"
                            },
                            {
                                "name": "CAOV3_dbgen1_JQ1_2_chromatin",
                                "well": "A2"
                            },
                            {
                                "name": "IMR32_dbgen1_JQ1_2_chromatin",
                                "well": "A3"
                            }
                        ]
                    }
                ]
            }
        ]
    }
    headers = {"Content-Type": "application/json"}
    response = requests.request("POST", url, json=payload, headers=headers)
    print(response.text)
    
make_new_plate()

{"uuid":"f5606be5-fe15-4acc-a986-ca1e9c8c372d","success":true}
